# Set

## 1. Maya server: run the follow command in the script editor in Maya

### open server
import maya.cmds as cmds

cmds.commandPort(n="localhost:12345")

### close server

cmds.commandPort(cl=True, n="localhost:12345")


In [1]:
import socket
import numpy as np

In [2]:
# connect to Maya server
HOST = '127.0.0.1'                 # Symbolic name meaning the local host
PORT = 12345              # Arbitrary non-privileged port

ADDR=(HOST,PORT)

client = socket.socket(socket.AF_INET, socket.SOCK_STREAM)
client.connect(ADDR)

In [3]:
def SendCommand(command:str):
    #client = socket.socket(socket.AF_INET, socket.SOCK_STREAM)
    #client.connect(ADDR)
    command = command.encode() # the command from external editor to maya

    MyMessage = command
    client.send(MyMessage)
    data = client.recv(1024) #receive the result info
    #client.close()
    #print(data)
    #ret = str(data.decode(encoding="ASCII"))
    ret = data.decode("utf-8")
    return ret

In [4]:
def SetCurrentTimeFrame(timeFrame : int):
    send_message = "currentTime -edit" + " " + str(timeFrame) + ";"
    recv_message = SendCommand(send_message)

In [6]:
# # Test one: set time frame
SetCurrentTimeFrame(0)

In [7]:
def SetObjectWorldTransform(object_name:str, location : list):
    '''
    Set world absolute location for object with location [x, y, z]
    '''
    send_message = "select -replace " + object_name + ";"
    send_message += "move -absolute " + str(location[0]) + " " + str(location[1]) + " " + str(location[2]) + ";"
    recv_message = SendCommand(send_message)

In [9]:
# # Test two: set object location
SetObjectWorldTransform("Main",[0,0,0])

In [10]:
def SetObjectLocalTransform(object_name:str, location : list):
    '''
    Set world absolute location for object with location [x,y] or [x, y, z]
    '''
    send_message = "select -replace " + object_name + ";" + "move -relative "
    for value in location:
        send_message +=  str(value) + " "
    
    send_message += ";"
    recv_message = SendCommand(send_message)

In [11]:
# # Test three: set object location transform
SetObjectLocalTransform("IKLeg_R", [1,2,2])

In [12]:
def SetObjectLocalRotation(object_name:str, rotation : list):
    '''
    Set world absolute location for object with rotation [x,y] or [x, y, z] in degree
    '''
    send_message = "select -replace " + object_name + ";" + "rotate -relative "
    for value in rotation:
        send_message +=  str(value) + "deg "
    
    send_message += ";"
    recv_message = SendCommand(send_message)

In [13]:
# # Test four: set object location transform
# SetObjectLocalRotation("FKShoulder_R", [20,30,40])

In [14]:
def SetCurrentKeyFrameForAttribute(object_name : str, attr_name : str):
    send_message = "select -r " + object_name + ";"
    send_message += "setKeyframe -at " + attr_name + ";"
    recv_message = SendCommand(send_message)

In [15]:
def SetCurrentKeyFrameForTransformAndRotation(object_name : str):
    send_message = "select -r " + object_name + ";"
    send_message += "setKeyframe -at translate;"
    send_message += "setKeyframe -at rotate;"
    recv_message = SendCommand(send_message)

In [16]:
# # Test five: Key Frame
# SetCurrentKeyFrameForTransformAndRotation("FKElbow_R")

In [17]:
def SetObjectAttribute(object_name : str, attr_name : str, value : float):
    send_message = "setAttr " + object_name + "." + attr_name + " " + str(value) + ";"
    recv_message = SendCommand(send_message)

In [18]:
# # Test six: set attribute
# SetObjectAttribute("ctrlEye_L","blink",5)

In [19]:
# Test seven: put together
SetCurrentTimeFrame(0)
SetCurrentKeyFrameForAttribute("ctrlEye_L", "blink")

SetCurrentTimeFrame(10)
SetObjectAttribute("ctrlEye_L","blink",10)
SetCurrentKeyFrameForAttribute("ctrlEye_L", "blink")

# Set body joint

In [20]:
#设置身体关节分为三类：根关节、FK关节和IK关节；根关节需要关注旋转和位置，FK只需要关心旋转，IK只需要关心位置（作为简化）

In [21]:
def SetBodyJoint(body_info : dict, keyframe = 0):
    SetCurrentTimeFrame(keyframe)
    
    for key in body_info.keys():
        if key == "Main":
            main_transform = body_info['Main']['world_transform']
            SetObjectWorldTransform("Main", main_transform)
            local_rotation = body_info['Main']['local_rotation']
            SetObjectLocalRotation('Main', local_rotation)
            SetCurrentKeyFrameForTransformAndRotation("Main")  
        elif key.startswith("FK") or key == "Neck_M":
            local_rotation = body_info[key]['local_rotation']
            SetObjectLocalRotation(key, local_rotation)
            SetCurrentKeyFrameForTransformAndRotation(key)
        else: #key.startswith("IK")
            local_transform = body_info[key]['local_transform']
            SetObjectLocalTransform(key, local_transform)
            SetCurrentKeyFrameForTransformAndRotation(key)
            
    print("Done!Body joints at frame " + str(keyframe))
    

In [22]:
body_info = {'Main': {'world_transform': [1.0, 1.0, 1.0],
  'local_transform': [0.0, 0.0, 0.0],
  'local_rotation': [0.0, 0.0, 0.0]},
 'FKRoot_M': {'world_transform': [0.0, 9.83, -0.18],
  'local_transform': [0.0, 0.0, 0.0],
  'local_rotation': [0.0, 0.0, 0.0]},
 'FKSpine1_M': {'world_transform': [-0.0, 11.57, -0.42],
  'local_transform': [0.0, 0.0, 0.0],
  'local_rotation': [0.0, 0.0, -0.55]},
 'FKChest_M': {'world_transform': [-0.0, 14.28, -0.56],
  'local_transform': [0.0, 0.0, 0.0],
  'local_rotation': [0.0, 0.0, 0.0]},
 'Neck_M': {'world_transform': [0.0, 16.63, -1.13],
  'local_transform': [2.42, 0.0, 0.0],
  'local_rotation': [0.0, 0.0, -0.0]},
 'FKShoulder_R': {'world_transform': [-2.24, 14.92, -0.98],
  'local_transform': [0.0, 0.0, 0.0],
  'local_rotation': [20, 30, 40]},
 'FKShoulder_L': {'world_transform': [2.24, 14.92, -0.98],
  'local_transform': [0.0, 0.0, 0.0],
  'local_rotation': [0.0, 0.0, 0.0]},
 'FKElbow_R': {'world_transform': [-5.21, 14.75, -1.09],
  'local_transform': [0.0, 0.0, 0.0],
  'local_rotation': [0.0, 0.0, 0.0]},
 'FKElbow_L': {'world_transform': [5.21, 14.75, -1.09],
  'local_transform': [0.0, 0.0, 0.0],
  'local_rotation': [0.0, 0.0, 0.0]},
 'FKWrist_R': {'world_transform': [-8.28, 14.74, -0.76],
  'local_transform': [0.0, 0.0, 0.0],
  'local_rotation': [0.0, 0.0, 0.0]},
 'FKWrist_L': {'world_transform': [8.28, 14.74, -0.76],
  'local_transform': [0.0, 0.0, 0.0],
  'local_rotation': [0.0, 0.0, 0.0]},
 'IKLeg_R': {'world_transform': [-1.47, 1.2, -0.71],
  'local_transform': [0.0, 0.0, 0.0],
  'local_rotation': [0.0, 0.0, 0.0]},
 'IKLeg_L': {'world_transform': [1.47, 1.2, -0.71],
  'local_transform': [1, 2, 0.0],
  'local_rotation': [0, 0, 0.0]}}

In [23]:
SetBodyJoint(body_info)

Done!Body joints at frame 0


# Set Finger Curl

In [24]:
def SetFingerCurl(finger_info : dict, keyframe = 0):
    SetCurrentTimeFrame(keyframe)
    
    fingers = ["thumb", "index", "middle", "ring", "pinky"]
    for finger in fingers:
        SetObjectAttribute("Fingers_R", finger + "Curl", finger_info["right_fingers"][finger + "_curl"][0])
        SetCurrentKeyFrameForAttribute("Fingers_R", finger + "Curl")
        SetObjectAttribute("Fingers_L", finger + "Curl", finger_info["left_fingers"][finger + "_curl"][0])
        SetCurrentKeyFrameForAttribute("Fingers_L", finger + "Curl")
            
    print("Done!Fingers at frame " + str(keyframe))
    

In [25]:
finger_info = {'right_fingers': {'thumb_curl': [0.0],
  'index_curl': [2.0],
  'middle_curl': [3.0],
  'ring_curl': [4.0],
  'pinky_curl': [0.0]},
 'left_fingers': {'thumb_curl': [0.0],
  'index_curl': [-1.0],
  'middle_curl': [0.0],
  'ring_curl': [-3.0],
  'pinky_curl': [0.0]}}

In [26]:
SetFingerCurl(finger_info)

Done!Fingers at frame 0


# Set Face

In [27]:
def SetFace(face_info : dict, keyframe = 0):
    SetCurrentTimeFrame(keyframe)
    
    #eye brow
    eye_brow_info = face_info['eye_brow']
    
    SetObjectAttribute("ctrlBrow_R", "translateX", eye_brow_info["eyebrow_r_x"][0])
    SetCurrentKeyFrameForAttribute("ctrlBrow_R", "translateX")
    
    SetObjectAttribute("ctrlBrow_R", "translateY", eye_brow_info["eyebrow_r_y"][0])
    SetCurrentKeyFrameForAttribute("ctrlBrow_R", "translateY")
    
    
    SetObjectAttribute("ctrlBrow_L", "translateX", eye_brow_info["eyebrow_l_x"][0])
    SetCurrentKeyFrameForAttribute("ctrlBrow_L", "translateX")
    
    SetObjectAttribute("ctrlBrow_R", "translateY", eye_brow_info["eyebrow_l_y"][0])
    SetCurrentKeyFrameForAttribute("ctrlBrow_L", "translateY")
    
    #eye
    eye_info = face_info['eye']
    
    SetObjectAttribute("ctrlEye_R", "translateX", eye_info["eye_r_x"][0])
    SetCurrentKeyFrameForAttribute("ctrlEye_R", "translateX")
    
    SetObjectAttribute("ctrlEye_R", "translateY", eye_info["eye_r_y"][0])
    SetCurrentKeyFrameForAttribute("ctrlEye_R", "translateY")
    
    SetObjectAttribute("ctrlEye_R", "blink", eye_info["eye_r_blink"][0])
    SetCurrentKeyFrameForAttribute("ctrlEye_R", "blink")
    
    
    SetObjectAttribute("ctrlEye_L", "translateX", eye_info["eye_l_x"][0])
    SetCurrentKeyFrameForAttribute("ctrlEye_L", "translateX")
    
    SetObjectAttribute("ctrlEye_L", "translateY", eye_info["eye_l_y"][0])
    SetCurrentKeyFrameForAttribute("ctrlEye_L", "translateY")
    
    SetObjectAttribute("ctrlEye_L", "blink", eye_info["eye_l_blink"][0])
    SetCurrentKeyFrameForAttribute("ctrlEye_L", "blink")
    
    #mouth
    mouth_info = face_info['mouth']
    
    SetObjectAttribute("ctrlMouth_M", "translateX", mouth_info["mouth_x"][0])
    SetCurrentKeyFrameForAttribute("ctrlMouth_M", "translateX")

    SetObjectAttribute("ctrlMouth_M", "translateY", mouth_info["mouth_y"][0])
    SetCurrentKeyFrameForAttribute("ctrlMouth_M", "translateY")
    
    #mouth corner
    mouth_corner_info = face_info['mouth_corner']
    
    SetObjectAttribute("ctrlMouthCorner_L", "translateY", mouth_corner_info["mouth_corner_L"][0])
    SetCurrentKeyFrameForAttribute("ctrlMouthCorner_L", "translateY")

    SetObjectAttribute("ctrlMouthCorner_R", "translateY", mouth_corner_info["mouth_corner_R"][0])
    SetCurrentKeyFrameForAttribute("ctrlMouthCorner_R", "translateY")
    
    print("Done!Face at frame " + str(keyframe))

In [28]:
face_info_frame_0 = {'eye_brow': {'eyebrow_r_x': [0.0],
  'eyebrow_r_y': [0.0],
  'eyebrow_l_x': [0.0],
  'eyebrow_l_y': [0.0]},
 'eye': {'eye_r_x': [0.0],
  'eye_r_y': [0.0],
  'eye_r_blink': [0.0],
  'eye_l_x': [0.0],
  'eye_l_y': [0.0],
  'eye_l_blink': [0.0]},
 'mouth': {'mouth_x': [0.0], 'mouth_y': [0.0]},
 'mouth_corner': {'mouth_corner_L': [0.0], 'mouth_corner_R': [0.0]}}

In [29]:
SetFace(face_info_frame_0, 0)

Done!Face at frame 0


In [30]:
face_info_frame_10 = {'eye_brow': {'eyebrow_r_x': [0.3],
  'eyebrow_r_y': [0.0],
  'eyebrow_l_x': [0.2],
  'eyebrow_l_y': [0.0]},
 'eye': {'eye_r_x': [0.0],
  'eye_r_y': [0.0],
  'eye_r_blink': [3],
  'eye_l_x': [0.0],
  'eye_l_y': [0.0],
  'eye_l_blink': [0.0]},
 'mouth': {'mouth_x': [0.2], 'mouth_y': [-0.5]},
 'mouth_corner': {'mouth_corner_L': [0.2], 'mouth_corner_R': [0.0]}}

In [31]:
SetFace(face_info_frame_10, 10)

Done!Face at frame 10


In [ ]:
# Now you can see a animation for face change in Maya